In [1]:
# third party
import numpy as np

# syft absolute
import syft as sy

# Verify Version

In [2]:
pip_info = !pip index versions syft
latest_deployed_version = pip_info[-1].split("LATEST: ")[-1].strip()

In [3]:
# this notebook should only be used to run the latest deployed version of syft
# the notebooks after this (1a/1b and 2), will test migrating from that latest version
print(
    f"latest deployed version: {latest_deployed_version}, installed version: {sy.__version__}"
)
# assert (
#     latest_deployed_version == sy.__version__
# ), f"{latest_deployed_version} does not match installed version {sy.__version__}"

latest deployed version: 0.9.1, installed version: 0.9.2-beta.1


# Launch Server

In [4]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    n_consumers=2,
    create_producer=True,
    port="auto",
)

Autoreload enabled
Starting test_upgradability server on 0.0.0.0:61547


INFO:     Will watch for changes in these directories: ['/Users/koen/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:61547 (Press CTRL+C to quit)
INFO:     Started reloader process [42876] using WatchFiles


Found `reset=True` in the launch configuration. Resetting the server...
Waiting for server to start

INFO:     Started server process [42880]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


 Done.


SyftInfo: You have launched a development server at http://0.0.0.0:61547.It is intended only for local use.

In [5]:
client = server.login(email="info@openmined.org", password="changethis")

Logged into <test_upgradability: High side Datasite> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [6]:
client.register(
    email="ds@openmined.org", name="John Doe", password="pw", password_verify="pw"
)

SyftSuccess: User 'John Doe' successfully registered! To see users, run `[your_client].users`

# Prepare some data to be migrated

In [7]:
client_ds = server.login(email="ds@openmined.org", password="pw")

Logged into <test_upgradability: High side Datasite> as <ds@openmined.org>


In [8]:
dataset = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=np.array([10, 11, 12, 13, 14]),
            data=np.array([[15, 16, 17, 18, 19] for _ in range(100_000)]),
            mock_is_real=True,
        )
    ],
)

client.upload_dataset(dataset)

Uploading: numpy-data: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.73it/s]


SyftSuccess: Dataset uploaded to 'test_upgradability'. To see the datasets uploaded by a client on this server, use command `[your_client].datasets`

In [9]:
data_low = client_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(datasite, data) -> float:
    # launch another job
    print("Computing mean...")
    return data

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [10]:
req = client_ds.code.request_code_execution(compute_mean)

In [11]:
client.requests[0].approve()

Approving request on change compute_mean for datasite test_upgradability


SyftSuccess: Request bf9ffc47ac6a411180d2628979ee50b3 changes applied

In [12]:
job = client_ds.code.compute_mean(data=data_low, blocking=False)

In [13]:
res = job.wait()

WARN: private key is based on server name: test_upgradability in dev_mode. Don't run this in production.
Document Store's SQLite DB path: /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/syft/21519e1e3e664b38a635dc951c293158/db/21519e1e3e664b38a635dc951c293158.sqlite
Action Store's SQLite DB path: /var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/syft/21519e1e3e664b38a635dc951c293158/db/21519e1e3e664b38a635dc951c293158.sqlite


06/09/24 11:45:13 FUNCTION LOG (5cf0ed8f222c4d99a985dcf41c0a0429): Computing mean...
06/09/24 11:45:14 FUNCTION LOG (5cf0ed8f222c4d99a985dcf41c0a0429): Computing mean...


In [14]:
res.get().shape

(100000, 5)

In [15]:
# todo: add more data

In [16]:
if server.server_type.value == "python":
    server.land()